# Backend

In [ ]:
import nltk
import numpy as np
import regex as re
import csv
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('punkt_tab')
stemmer = SnowballStemmer('spanish')

In [ ]:
### 1. Preparación

In [ ]:
# abrir el csv
list = [] # Guarda las canciones
with open('spotify_songs.csv', newline='', encoding='utf-8') as file:
    spamreader = csv.reader(file, delimiter=',', quotechar='"')
    for row in spamreader:
        string = ""
        string = row[1] + " " + row[2] + " " + row[3]
        # for campo in row:
        #     string += campo + " "
        # print(string)
        list.append(string)

for i in range(1, len(list)):
    print(list[i])

# print(len(list))


### 1- Preprocesamiento

In [ ]:
# Tokenizar, filtrar stopwords, eliminar signos, stemming

with open("stoplist.txt", encoding="utf-8") as file:
    stoplist = [line.rstrip().lower() for line in file]
    
def preprocesamiento(texto,stemming=True):
  words = []
  # 1- convertir a minusculas
  texto = texto.lower()
  # 2- eliminar signos con regex
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # 3- tokenizar
  words = nltk.word_tokenize(texto, language='spanish')
  # 3- eliminar stopwords
  words = [word for word in words if word not in stoplist]
  # 4- Aplicar reduccion de palabras (stemming)
  if stemming:        
      words = [stemmer.stem(word) for word in words]
  return words